In [1]:
import ee
ee.Authenticate()
# ee.Initialize()

d:\wangari\maathai\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)



Successfully saved authorization token.


In [2]:
ee.Initialize(project="ee-oderaisaack2")

In [9]:
# makueni = makueni_point.buffer(500).bounds()
mulooni = ee.Geometry.Point([37.49, -1.60]).buffer(2000).bounds()
mavindu = ee.Geometry.Point([37.44, -1.66]).buffer(2000).bounds()
kivale  = ee.Geometry.Point([37.45, -1.62]).buffer(2000).bounds()
katende = ee.Geometry.Point([37.45, -1.698]).buffer(2000).bounds()

chyulu = ee.Geometry.Point([37.8800, -2.6800]).buffer(5000).bounds()

# kalumbi = ee.Geometry.Point([28.600, -1.95]).buffer(2000).bounds()
kibwezi = ee.Geometry.Point([37.91, -2.43]).buffer(6000).bounds()
makuli = ee.Geometry.Point([37.4986, -1.8239]).buffer(17000).bounds()
kilungu = ee.Geometry.Point([37.33252, -1.79457]).buffer(8000).bounds()


forests = {
    "mulooni": mulooni,
    "mavindu": mavindu,
    "kivale": kivale,
    "katende": katende,
    "chyulu": chyulu,
    "kibwezi": kibwezi,
    "makuli": makuli,
    "kilungu": kilungu
}

In [10]:
sentinel_collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2015-01-01', '2025-11-22')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
)

In [ ]:
def extract_s1_safe(image, region, forest_name):
    all_bands = ['VV', 'VH', 'HH', 'HV']
    available = image.bandNames()

    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=10,
        bestEffort=True,
        maxPixels=1e12
    )
    
    feature = {
        "forest": forest_name,
        "date": ee.Date(image.get("system:time_start")).format("YYYY-MM-dd")
    }

    for b in all_bands:
        feature[b] = ee.Algorithms.If(
            available.contains(b),
            stats.get(b),
            None
        )

    return ee.Feature(None, feature)


In [ ]:
# result = sentinel_collection.map(extract_s1_safe)
all_fc = []

for forest_name, region in forests.items():
    fc = sentinel_collection \
        .filterBounds(region) \
        .map(lambda img: extract_s1_safe(img, region, forest_name))
    
    all_fc.append(fc)

final_fc = ee.FeatureCollection(all_fc).flatten()


In [15]:
task = ee.batch.Export.table.toDrive(
    collection=final_fc,
    description="makss",
    fileFormat="CSV",
    fileNamePrefix="SentinelMakueni"  
)

In [16]:
task.start()
print("Export started. Check Tasks in Earth Engine.")

Export started. Check Tasks in Earth Engine.
